In [16]:
import os
from mindee import Client, documents
import pandas as pd

# Init a new client
mindee_client = Client(api_key="Api_key")

# Folder path containing the receipts
folder_path = "C:/Users/????/Documents/GitHub/Reciept_Scrapper-/Receipts"

# List to store all the data frames
dfs = []

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".jpg"):  # Process only image files (modify file extension if necessary)
        file_path = os.path.join(folder_path, file_name)
        df = process_receipt(file_path)
        dfs.append(df)

# Concatenate all the data frames
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined data frame to a CSV file
combined_df.to_csv('receipts.csv', index=False)


In [13]:
def process_receipt(file_path):
    # Load a file from disk
    input_doc = mindee_client.doc_from_path(file_path)

    # Parse the document as an invoice by passing the appropriate type
    api_response = input_doc.parse(documents.TypeReceiptV5)

    quantity = []
    unit_price = []
    description = []
    total_amount = []
    date = []
    time_of_p = []
    store = []

    for line_item in api_response.document.line_items:
        description.append(line_item.description)
        quantity.append(line_item.quantity)
        unit_price.append(line_item.unit_price)
        total_amount.append(line_item.total_amount)
        date.append(api_response.document.date.value)
        time_of_p.append(api_response.document.time.value)
        store.append(api_response.document.supplier_name.value)

    line_items = [description, quantity, unit_price, total_amount, date, time_of_p, store]

    # Transpose the list of lists
    line_items = list(zip(*line_items))

    # Create the data frame
    df = pd.DataFrame(line_items, columns=['Description', 'Quantity', 'Unit Price', 'Total Amount', 'Date', 'Time', 'Store'])

    return df